## Merge different datasets

Some images and tweets of PMA were downloaded and the data were stored in a PGSQL database, and a different part of PMA was just stored as json files on a disk. This notebook aims to merge all this data in a database. 

In [1]:
import concurrent
import pandas
import os
import json
from tqdm import tqdm
import psycopg2

In [2]:
TWEETS = r"/home/tyra/Documents/CERES/PMA/images_3"
HASHS = r"/home/tyra/Documents/CERES/PMA/images_3/sha1.json"
AVERAGE_HASHS = r'/home/tyra/Documents/CERES/PMA/hashs/sha1_to_ahash_8.json'

In [3]:
with open(HASHS, 'r') as f:
    hashs = json.load(f)

## Merge Data
Insert data collected in json format to the postgresql base

In [5]:
def insert_tweet(path, bar):
    with open(path, 'r') as f:
        tweet = json.load(f)
    id_ = tweet['id']
    full_text = tweet['text']
    source = json.dumps(tweet)
    # update the base with the tweet
    cursor.execute("INSERT INTO pma_tweets (id, full_text, source) VALUES (%s, %s, %s)", (id_, full_text, source))
    if tweet.get('attachments', {}).get('media_keys', False):
        for key in tweet['attachments']['media_keys']:
            hash_ = hashs.get(key, None)
            if hash_:
                hash_, extension = hash_.split('.')
                # print(key, extension, hash_, id_)
                cursor.execute("INSERT INTO pma_media (id, extension, sha1, tweet_id) VALUES (%s, %s, %s, %s)", 
                              (key, extension, hash_, id_))
                # update the base with the image_key + the sha1 + tweet_id + extension
    conn.commit()
    bar.update(1)

In [24]:
def delete_tweets():
    conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="edwin007")
    cursor = conn.cursor()

    for tweet in os.listdir(TWEETS):
        with open(os.path.join(TWEETS, tweet), 'r') as f:
            tweet = json.load(f)
        try:
            id_ = tweet['id']
        except KeyError:
            pass
        cursor.execute("DELETE from pma_tweets WHERE id=%s", (id_,))
        if tweet.get('attachments', {}).get('media_keys', False):
            for key in tweet['attachments']['media_keys']:
                cursor.execute("DELETE from pma_media WHERE id=%s", (key,))
    conn.commit()

In [ ]:
delete_tweets()

In [6]:
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="edwin007")
cursor = conn.cursor()

tweets = os.listdir(TWEETS)
with tqdm(total=len(tweets)) as pbar:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        [executor.submit(insert_tweet, os.path.join(TWEETS, tweet), pbar) for tweet in tweets]    

 11%|███▉                               | 11964/107692 [00:23<03:06, 512.23it/s]


## Add Average Hashs to Data

Add the average hashs of the image to the fred_pma_media table

In [3]:
with open(AVERAGE_HASHS, 'r') as f:
    hashs = json.load(f)

# file in the form {sha1: ahash}

In [4]:
def update_ahash(sha1, ahash, pbar):
    cursor.execute("UPDATE pma_media SET average_hash_8=%s WHERE sha1=%s", (ahash, sha1))
    conn.commit()
    pbar.update(1)

In [5]:
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="edwin007")
cursor = conn.cursor()

with tqdm(total=len(hashs)) as pbar:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        [executor.submit(update_ahash, sha1, hashs[sha1], pbar) for sha1 in hashs] 

100%|███████████████████████████████████| 64445/64445 [1:12:04<00:00, 14.90it/s]
